## Model description

https://easychair.org/publications/paper/1ff2

Three-dimensional ODE of the plant without discrete-time PI-controller:

$$
   \left\{ \begin{array}{lcl}
   \dot{I} & = & \frac{1}{L}\cdot\left( \left(K_P\cdot\left(x_0-x\right)+K_I\cdot x_c \right) - I\cdot\left(R - \frac{K^2}{d_{rot}}\right)\right) \\
   \dot{x} & = & \frac{K}{i\cdot d_{rot}}\cdot I \\
   \dot{x_c} & = & x_0-x \end{array}
   \right.
$$

In [ ]:
using Reachability, TaylorIntegration, MathematicalSystems, HybridSystems, SparseArrays
include("helper.jl")

In [ ]:
function EMBrake_hibrid()
    # the state variables are:
    # [I, x, T, xe, xc]

    # model's constants
    L       = 1.e-3;
    KP      = 10000.;
    KI      = 1000.;
    R       = 0.3;
    K       = 0.02;
    drot    = 0.1;
    i       = 113.1167;
    Tsample = 1.E-4;
    
    EMbrake = LightAutomaton(1)

    # Braking mode: x' =  Ax + b
    A = [-(R+K^2/drot)/L 0   0   KP/L  KI/L; 
           K/i/drot      0   0   0      0;
           0             0   0   0      0;
           0             0   0   0      0;
           0             0   0   0      0]
    A = sparse(A)
    b = [0, 0, 1.0, 0, 0]
    b = sparse(b)
    X = HalfSpace([0.0, 0.0, 1.0, 0.0, 0.0], Tsample) # invariant
    brake = ConstrainedAffineContinuousSystem(A, b, X)
    modes = [brake]

    # transition (self-loop)
    add_transition!(EMbrake, 1, 1, 1)
    x0 = 0.05
    A = sparse([1, 2, 4, 5, 5], [1, 2, 2, 2, 5], [1., 1., -1., -Tsample, 1.], 5, 5);
    b = sparsevec([4, 5], [x0, Tsample*x0], 5)
    guard = HalfSpace([0.0, 0.0, -1.0, 0.0, 0.0], -Tsample); # T >= Tsample
    resetmaps = [ConstrainedAffineMap(A, b, guard)]
    switchings = [AutonomousSwitching()]

    H = HybridSystem(EMbrake, modes, resetmaps, switchings)
end

### Reachability settings

In [ ]:
I₀  = Singleton([0.0]); #Interval(0., 0.);
x₀  = Singleton([0.0]); #Interval(0., 0.);
T₀  = Singleton([0.0]); #Interval(0., 0.);
xe₀ = Singleton([0.0]); #Interval(0., 0.);
xc₀ = Singleton([0.0]); #Interval(0., 0.);

X₀ = I₀ × x₀ × T₀ × xe₀ × xc₀;
#X₀ = X₀ ⊕ BallInf(zeros(dim(X₀)), 0.0001);

### Results

In [ ]:
H = EMBrake_hibrid()
P = InitialValueProblem(H, [(1, X₀)]);

In [ ]:
alg = BFFPSV18(:δ=>1e-4)
# opD = LazyDiscretePost(:check_invariant_intersection => true)
sol = solve(P, Options(:T=>0.4, :verbosity=>2, :max_jumps=>3), alg);